HARIPRASAD SIVAPATHAM ANAND 

X2022gmv

In [29]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [4]:
auc_values = pd.read_csv("./AUC_VALUES.csv")

In [5]:
required_columns = []
for index,row in auc_values.iterrows():
    if(row["AUC_VALUES"]>0.75):
        required_columns.append(row["Columns"])

In [50]:
auc_values[0:20]

,Columns,AUC_VALUES
0,Assay_Id,0.854496
1,BCUT2D_CHGHI,0.791767
2,BCUT2D_MRHI,0.791709
3,BCUT2D_LOGPHI,0.791671
4,BCUT2D_CHGLO,0.791207
5,Chi2v,0.790936
6,BCUT2D_MRLOW,0.790271
7,BCUT2D_LOGPLOW,0.790132
8,Chi3v,0.789785
9,Chi2n,0.789624


In [6]:
train_data = pd.read_csv("./detail_smile_data_train.csv")

In [7]:
required_columns

['Assay_Id',
 'BCUT2D_CHGHI',
 'BCUT2D_MRHI',
 'BCUT2D_LOGPHI',
 'BCUT2D_CHGLO',
 'Chi2v',
 'BCUT2D_MRLOW',
 'BCUT2D_LOGPLOW',
 'Chi3v',
 'Chi2n',
 'BCUT2D_MWLOW',
 'MaxAbsEStateIndex',
 'MaxEStateIndex',
 'Chi3n',
 'Chi1v',
 'Chi4v',
 'qed',
 'Chi4n',
 'Chi1n',
 'MolMR',
 'MolLogP',
 'BertzCT',
 'MinEStateIndex',
 'Chi0v',
 'LabuteASA',
 'Kappa3',
 'Chi0n',
 'MinAbsPartialCharge',
 'MaxPartialCharge',
 'MinAbsEStateIndex',
 'Kappa2',
 'ExactMolWt',
 'MolWt',
 'MinPartialCharge',
 'MaxAbsPartialCharge',
 'BalabanJ',
 'Kappa1',
 'Ipc']

In [8]:
# Replacing missing data with mean
for col in train_data:
    mean = train_data[col].mean()
    train_data[col] = train_data[col].fillna(mean)

In [9]:
train_data

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Assay_Id,label
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,1644,2
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,4,0,2451,2
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,12,0,1384,2
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,16,2
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,1856,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75378,11.460021,-3.868472,11.460021,0.053611,0.712426,230.245,220.165,230.036128,82,0,...,0,0,0,0,0,0,0,0,33,2
75379,5.928972,-2.841623,5.928972,0.082346,0.720533,313.747,296.611,313.041677,104,0,...,0,0,0,0,0,0,0,0,1632,1
75380,4.975926,0.848333,4.975926,0.848333,0.596343,167.258,162.218,166.986341,50,0,...,0,0,0,1,0,0,0,0,1373,1
75381,10.241948,0.324028,10.241948,0.324028,0.519485,128.215,112.087,128.120115,54,0,...,0,0,0,0,0,0,0,0,2,2


In [53]:
X = train_data[required_columns[0:2]]
y = train_data["label"]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'n_estimators': [100,150, 200,250],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [30]:
CV_rfc = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv= 5,verbose=2,n_jobs=6)
CV_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=6,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [100, 150, 200, 250]},
             verbose=2)

In [32]:
CV_rfc.best_score_
print(CV_rfc.best_params_)

{'criterion': 'gini', 'max_depth': 8, 'max_features': 'auto', 'n_estimators': 100}


In [33]:
model = RandomForestClassifier(criterion="gini",max_depth=8,max_features="auto",n_estimators=100)

In [ ]:
CV_rfc = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv= 5,v)
CV_rfc.fit(x_train, y_train)

In [54]:
model.fit(X,y)

RandomForestClassifier(max_depth=8, max_features='auto')

In [15]:
# model.score(X_test,y_test)

0.8399129788814602

In [35]:
test_data = pd.read_csv("./detail_smile_data_test.csv")

In [36]:
test_data_clean = pd.read_csv("./data/test_data_new")

In [37]:
test_data["Assay_Id"] = test_data_clean.assay_ids

In [38]:
for col in test_data:
    mean = test_data[col].mean()
    test_data[col] =  test_data[col].fillna(mean)

In [55]:
predict = model.predict(test_data[required_columns[0:2]])

In [56]:
sub_file = pd.read_csv("./data/sample_submission.csv")

In [57]:
sub_file["Predicted"] = predict

In [58]:
sub_file.to_csv("Submission_1_feb_3.csv",index=False)